In [172]:
import pandas as pd
import pymongo

In [173]:
client = pymongo.MongoClient()

result_list = list(client['Asic_machine_profit']['Finland'].find({}))

In [174]:
full_df = pd.DataFrame()

for result in result_list:
    temp_df = pd.DataFrame(result["data"])
    temp_df["machine_name"] = result["machine_name"]
    full_df = pd.concat([full_df, temp_df])

In [175]:
full_df

,coin,name,type,algorithm,network_hashrate,difficulty,reward_block,reward_unit,profit_hourly,profit_daily,profit_monthly,hourly_electricity_cost,daily_electricity_cost,monthly_electricity_cost,final_profit_hourly,final_profit_daily,final_profit_monthly,machine_name
0,2MINERS CLO,2Miners,pool,ethash,0.000000e+00,0.000000e+00,0.000000,CLO,1.956338e+00,4.695211e+01,1.408563e+03,0.462936,11.110470,333.314102,1.493402,35.841637,1075.249099,iPollo_V1
1,2MINERS ETH,2Miners,pool,ethash,0.000000e+00,0.000000e+00,0.000000,ETH,2.996900e+00,7.192560e+01,2.157768e+03,0.462936,11.110470,333.314102,2.533964,60.815125,1824.453752,iPollo_V1
2,2MINERS ETP,2Miners,pool,ethash,0.000000e+00,0.000000e+00,0.000000,ETP,1.170043e-10,2.808103e-09,8.424309e-08,0.462936,11.110470,333.314102,-0.462936,-11.110470,-333.314102,iPollo_V1
3,2MINERS EXP,2Miners,pool,ethash,0.000000e+00,0.000000e+00,0.000000,EXP,8.665104e-01,2.079625e+01,6.238875e+02,0.462936,11.110470,333.314102,0.403574,9.685778,290.573350,iPollo_V1
4,AKA,Akroma,coin,ethash,1.120049e+09,1.557988e+10,5.000000,AKA,1.754899e+00,4.211759e+01,1.263528e+03,0.462936,11.110470,333.314102,1.291963,31.007115,930.213450,iPollo_V1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,VEIL,VEIL,coin,sha-256,0.000000e+00,0.000000e+00,30.000000,VEIL,0.000000e+00,0.000000e+00,0.000000e+00,0.955739,22.937745,688.132339,-0.955739,-22.937745,-688.132339,Bitfury_B8
28,VIABTC BTC,ViaBTC,pool,sha-256,0.000000e+00,0.000000e+00,0.000000,BTC,1.913466e-01,4.592318e+00,1.377695e+02,0.955739,22.937745,688.132339,-0.764393,-18.345427,-550.362801,Bitfury_B8
29,VLS,Veles,coin,sha-256,4.527518e+11,1.901941e+05,0.004714,VLS,8.590638e-04,2.061753e-02,6.185259e-01,0.955739,22.937745,688.132339,-0.954880,-22.917127,-687.513813,Bitfury_B8
30,ZP SHA-256,Zergpool,pool,sha-256,0.000000e+00,0.000000e+00,0.000000,BTC,2.105765e-01,5.053836e+00,1.516151e+02,0.955739,22.937745,688.132339,-0.745163,-17.883909,-536.517273,Bitfury_B8


In [176]:
sort=list({'_id': -1}.items())
limit=1
result = client['Shipments']['international_routes'].find(
    sort=sort,
    limit=limit
)
shipment_df = pd.DataFrame(result[0]["data"])
shipment_df

,class,size(inch cube),weight (lb),from,to,price ($)
0,150%,1267.128499,27.623248,China,Finland,653
1,150%,1267.128499,27.623248,China,New York,275
2,150%,1267.128499,27.623248,Germany,Finland,111
3,150%,1267.128499,27.623248,Germany,New York,248
4,150%,1267.128499,27.623248,Texas,Finland,194
5,150%,1267.128499,27.623248,Texas,New York,39
6,125%,1055.940415,23.019374,China,Finland,569
7,125%,1055.940415,23.019374,China,New York,248
8,125%,1055.940415,23.019374,Germany,Finland,111
9,125%,1055.940415,23.019374,Germany,New York,220


In [177]:
sort=list({'_id': -1}.items())
limit=1
result = client['Crypto-mining']['ASICS-PoW-final'].find(
    sort=sort,
    limit=limit
)
market_df = pd.DataFrame(result[0]["data"])
market_df["machine_name"] = market_df["Manufacturer"] + " " + market_df["Model"]
machine_info_df = market_df
market_df = market_df[["machine_name",'available_stores']]
market_df

,machine_name,available_stores
0,iPollo V1,"[{'store_name': 'CoinMining Central', 'url': '..."
1,Jasminer X4,"[{'store_name': 'Safety Miner', 'url': 'https:..."
2,Bitmain Antminer E9 (2.4Gh),"[{'store_name': 'CoinMining Central', 'url': '..."
3,Innosilicon A11 Pro ETH (1500Mh),"[{'store_name': 'NHASH', 'url': 'https://www.n..."
4,iPollo V1 Classic,"[{'store_name': 'Belay Mining', 'url': 'https:..."
...,...,...
256,Spondoolies SPx36,"[{'store_name': 'CoinMining Central', 'url': '..."
257,Bitfury Tardis,None
258,Bitfily Snow Panther A1,None
259,Pantech WX6,None


In [178]:
market_df.iloc[0,1][1]

{'store_name': 'AKMiner',
 'url': 'https://www.akminer.com/all-products/ipollo-v1-3600mh-eth-miner-most-powerful-eth-miner-for-2022?wpam_id=1',
 'price': '$20,299.00',
 'country': 'China',
 'stock': 'In stock(3 \ndays\n)',
 'isFreeShipping': True}

## Do two different dataframes one for us one for Finland to determine the cheapest machine for each location --> US calculate location is New York

In [179]:
fin_market_df = market_df
fin_market_df = fin_market_df[~fin_market_df["available_stores"].isna()]
us_market_df = market_df
us_market_df = us_market_df[~us_market_df["available_stores"].isna()]

In [180]:
fin_market_df

,machine_name,available_stores
0,iPollo V1,"[{'store_name': 'CoinMining Central', 'url': '..."
1,Jasminer X4,"[{'store_name': 'Safety Miner', 'url': 'https:..."
2,Bitmain Antminer E9 (2.4Gh),"[{'store_name': 'CoinMining Central', 'url': '..."
3,Innosilicon A11 Pro ETH (1500Mh),"[{'store_name': 'NHASH', 'url': 'https://www.n..."
4,iPollo V1 Classic,"[{'store_name': 'Belay Mining', 'url': 'https:..."
...,...,...
243,StrongU STU-U2,"[{'store_name': 'CoinMining Central', 'url': '..."
244,MicroBT Whatsminer M10S,"[{'store_name': 'Bitcoin Merch', 'url': 'https..."
251,Todek Toddminer C1 PRO,"[{'store_name': 'CoinMining Central', 'url': '..."
255,Hummer Miner Mars H1,"[{'store_name': 'Mining Store 5111', 'url': 'h..."


In [181]:
countries = []
for x in range(fin_market_df.shape[0]):
    machine_market_data = fin_market_df.iloc[x, 1]
    for market in machine_market_data:
        print(market)
        countries.append(market["country"])
        print()


{'store_name': 'CoinMining Central', 'url': 'https://coinminingcentral.com/collections/ipollo-miners/products/ipollo-v1-mini-ethereum-miner-3-6-gh-s?ref=uybdgyQD&ref=uybdgyQD', 'price': '$22,273.98', 'country': 'United Kingdom', 'stock': 'In stock', 'isFreeShipping': True}

{'store_name': 'AKMiner', 'url': 'https://www.akminer.com/all-products/ipollo-v1-3600mh-eth-miner-most-powerful-eth-miner-for-2022?wpam_id=1', 'price': '$20,299.00', 'country': 'China', 'stock': 'In stock(3 \ndays\n)', 'isFreeShipping': True}

{'store_name': 'Mining Store 5111', 'url': 'https://www.5111.store/product/ipollo-v1-3400m/?wpam_id=1', 'price': '$20,366.00', 'country': 'Hong Kong', 'stock': 'In stock', 'isFreeShipping': True}

{'store_name': 'NHASH', 'url': 'https://www.nhash.net/miners/94129954756763648?ref=3yrntphlqn', 'price': '$20,499.00', 'country': 'Hong Kong', 'stock': 'In stock', 'isFreeShipping': False}

{'store_name': 'Belay Mining', 'url': 'https://belaymining.com/collections/ethereum-miner/prod

In [182]:
inv_countries = list(set(countries))
inv_countries

['Canada',
 'Italia',
 'Israel',
 'Hong Kong',
 'Honk Kong',
 'Germany',
 'United States',
 'United Kingdom',
 'China']

In [183]:
country_mapping = {
    "EU": [inv_countries[1], inv_countries[7], inv_countries[5]],
    "US": [inv_countries[6], inv_countries[0]],
    "ASIA": [inv_countries[2], inv_countries[3], inv_countries[8], inv_countries[4]]
}
country_mapping

{'EU': ['Italia', 'United Kingdom', 'Germany'],
 'US': ['United States', 'Canada'],
 'ASIA': ['Israel', 'Hong Kong', 'China', 'Honk Kong']}

## Best markets for each machine for Finland

In [184]:
machine_info_df

,Manufacturer,Model,Release,Size(mm),Weight(g),Noise level,Fan(s),Power(w),Interface,Memory,...,Chip count,Chip boards,Chip name,Chip size,Cooling,Wires,Rack format,Processor,Operating system,machine_name
0,iPollo,V1,June 2022,"{'x': 314.0, 'y': 194.0, 'z': 290.0}",13000.0,70db,4,3100.0,Ethernet,6G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iPollo V1
1,Jasminer,X4,November 2021,NaN,NaN,75db,4,1200.0,Ethernet,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jasminer X4
2,Bitmain,Antminer E9 (2.4Gh),July 2022,"{'x': 195.0, 'y': 290.0, 'z': 400.0}",14200.0,75db,4,1920.0,Ethernet,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bitmain Antminer E9 (2.4Gh)
3,Innosilicon,A11 Pro ETH (1500Mh),November 2021,NaN,NaN,75db,4,2350.0,Ethernet,8Gb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Innosilicon A11 Pro ETH (1500Mh)
4,iPollo,V1 Classic,June 2022,"{'x': 314.0, 'y': 194.0, 'z': 290.0}",13000.0,70db,4,1240.0,Ethernet,3.8G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iPollo V1 Classic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Spondoolies,SPx36,September 2018,"{'x': 595.0, 'y': 434.0, 'z': 88.0}",19500.0,75db,4,4400.0,Ethernet,NaN,...,150,NaN,NaN,NaN,NaN,NaN,2U,NaN,NaN,Spondoolies SPx36
257,Bitfury,Tardis,November 2018,"{'x': 264.0, 'y': 483.0, 'z': 591.0}",37000.0,80db,6,6300.0,"Ethernet 100BASE-TX, IEEE 802.3u",NaN,...,1024,8,Bitfury Clarke,14nm,NaN,NaN,6U,NaN,NaN,Bitfury Tardis
258,Bitfily,Snow Panther A1,January 2018,"{'x': 275.0, 'y': 483.0, 'z': 599.0}",40000.0,82db,6 x 12038,5400.0,Ethernet,NaN,...,576,6,BF16BTC,16nm,NaN,NaN,6U,NaN,NaN,Bitfily Snow Panther A1
259,Pantech,WX6,January 2018,NaN,NaN,72db,2,5000.0,Ethernet,NaN,...,416,8,Bitfury,16nm,NaN,NaN,NaN,NaN,NaN,Pantech WX6


In [185]:
x_list = []
y_list = []
z_list = []
for x in range(machine_info_df.shape[0]):
    size = machine_info_df.iloc[x, 3]
    if str(size) != "nan":
        x_list.append(size["x"])
        y_list.append(size["y"])
        z_list.append(size["z"])
    else:
        x_list.append(0)
        y_list.append(0)
        z_list.append(0)

In [186]:
machine_info_df["width"] = x_list
machine_info_df["length"] = z_list
machine_info_df["height"] = y_list
machine_info_df["total_size"] = machine_info_df["width"] * machine_info_df["length"] * machine_info_df["height"]

## cube mm to cube inch divide value by 16390
## g to lb for an approximate result, divide the mass value by 453.6

In [187]:
machine_info_df["cube_inch"] = machine_info_df["total_size"] / 16390
machine_info_df["lb"] = machine_info_df["Weight(g)"] / 453.6
machine_info_df

,Manufacturer,Model,Release,Size(mm),Weight(g),Noise level,Fan(s),Power(w),Interface,Memory,...,Rack format,Processor,Operating system,machine_name,width,length,height,total_size,cube_inch,lb
0,iPollo,V1,June 2022,"{'x': 314.0, 'y': 194.0, 'z': 290.0}",13000.0,70db,4,3100.0,Ethernet,6G,...,NaN,NaN,NaN,iPollo V1,314.0,290.0,194.0,17665640.0,1077.830384,28.659612
1,Jasminer,X4,November 2021,NaN,NaN,75db,4,1200.0,Ethernet,5G,...,NaN,NaN,NaN,Jasminer X4,0.0,0.0,0.0,0.0,0.000000,NaN
2,Bitmain,Antminer E9 (2.4Gh),July 2022,"{'x': 195.0, 'y': 290.0, 'z': 400.0}",14200.0,75db,4,1920.0,Ethernet,NaN,...,NaN,NaN,NaN,Bitmain Antminer E9 (2.4Gh),195.0,400.0,290.0,22620000.0,1380.109823,31.305115
3,Innosilicon,A11 Pro ETH (1500Mh),November 2021,NaN,NaN,75db,4,2350.0,Ethernet,8Gb,...,NaN,NaN,NaN,Innosilicon A11 Pro ETH (1500Mh),0.0,0.0,0.0,0.0,0.000000,NaN
4,iPollo,V1 Classic,June 2022,"{'x': 314.0, 'y': 194.0, 'z': 290.0}",13000.0,70db,4,1240.0,Ethernet,3.8G,...,NaN,NaN,NaN,iPollo V1 Classic,314.0,290.0,194.0,17665640.0,1077.830384,28.659612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Spondoolies,SPx36,September 2018,"{'x': 595.0, 'y': 434.0, 'z': 88.0}",19500.0,75db,4,4400.0,Ethernet,NaN,...,2U,NaN,NaN,Spondoolies SPx36,595.0,88.0,434.0,22724240.0,1386.469799,42.989418
257,Bitfury,Tardis,November 2018,"{'x': 264.0, 'y': 483.0, 'z': 591.0}",37000.0,80db,6,6300.0,"Ethernet 100BASE-TX, IEEE 802.3u",NaN,...,6U,NaN,NaN,Bitfury Tardis,264.0,591.0,483.0,75359592.0,4597.900671,81.569665
258,Bitfily,Snow Panther A1,January 2018,"{'x': 275.0, 'y': 483.0, 'z': 599.0}",40000.0,82db,6 x 12038,5400.0,Ethernet,NaN,...,6U,NaN,NaN,Bitfily Snow Panther A1,275.0,599.0,483.0,79562175.0,4854.312081,88.183422
259,Pantech,WX6,January 2018,NaN,NaN,72db,2,5000.0,Ethernet,NaN,...,NaN,NaN,NaN,Pantech WX6,0.0,0.0,0.0,0.0,0.000000,NaN


In [188]:
def check_machine_category(machine_name): # Checks what is category of size the machines is closest to calculate shipment cost
    machine_df = machine_info_df[(machine_info_df["machine_name"] == machine_name)]
    custom_var_number = (machine_df["cube_inch"] * machine_df["lb"]).iloc[0]
    category_custom_var_numbers = [35002, 24307, 15556, 8750, 3889]
    difference_list = []

    for index, number in enumerate(category_custom_var_numbers):
        difference = abs(number - custom_var_number)
        difference_list.append(difference)

    # print(difference_list)
    smallest_numbers_index = difference_list.index(min(difference_list))
    category_dict = {
        0: "150%",
        1: "125%",
        2: "100%",
        3: "75%",
        4: "50%"
    }
    return category_dict[smallest_numbers_index]

In [189]:
check_machine_category("iPollo V1")

'150%'

In [190]:
shipment_df[shipment_df["to"] == "Finland"]
shipment_df["size(inch cube)"] * shipment_df["weight (lb)"]

0     35002.205079
1     35002.205079
2     35002.205079
3     35002.205079
4     35002.205079
5     35002.205079
6     24307.086860
7     24307.086860
8     24307.086860
9     24307.086860
10    24307.086860
11    24307.086860
12    15556.535591
13    15556.535591
14    15556.535591
15    15556.535591
16    15556.535591
17    15556.535591
18     8750.551270
19     8750.551270
20     8750.551270
21     8750.551270
22     8750.551270
23     8750.551270
24     3889.133898
25     3889.133898
26     3889.133898
27     3889.133898
28     3889.133898
29     3889.133898
dtype: float64

In [191]:
fin_market_df

,machine_name,available_stores
0,iPollo V1,"[{'store_name': 'CoinMining Central', 'url': '..."
1,Jasminer X4,"[{'store_name': 'Safety Miner', 'url': 'https:..."
2,Bitmain Antminer E9 (2.4Gh),"[{'store_name': 'CoinMining Central', 'url': '..."
3,Innosilicon A11 Pro ETH (1500Mh),"[{'store_name': 'NHASH', 'url': 'https://www.n..."
4,iPollo V1 Classic,"[{'store_name': 'Belay Mining', 'url': 'https:..."
...,...,...
243,StrongU STU-U2,"[{'store_name': 'CoinMining Central', 'url': '..."
244,MicroBT Whatsminer M10S,"[{'store_name': 'Bitcoin Merch', 'url': 'https..."
251,Todek Toddminer C1 PRO,"[{'store_name': 'CoinMining Central', 'url': '..."
255,Hummer Miner Mars H1,"[{'store_name': 'Mining Store 5111', 'url': 'h..."


In [192]:
def calculate_shipment(data, destination_country):
    if data["isFreeShipping"] is True:
        return 0
    ship_df = shipment_df[(shipment_df["to"] == "Finland") & (shipment_df["class"] == data["size_category"])]
    if data["continent"] == "EU":
        ship_df = ship_df[ship_df["from"] == "Germany"]
    elif data["continent"] == "US":
        ship_df = ship_df[ship_df["from"] == "Texas"]
    elif data["continent"] == "ASIA":
        ship_df = ship_df[ship_df["from"] == "China"]
    return ship_df.iloc[0, 5]

In [193]:
shipment_df

,class,size(inch cube),weight (lb),from,to,price ($)
0,150%,1267.128499,27.623248,China,Finland,653
1,150%,1267.128499,27.623248,China,New York,275
2,150%,1267.128499,27.623248,Germany,Finland,111
3,150%,1267.128499,27.623248,Germany,New York,248
4,150%,1267.128499,27.623248,Texas,Finland,194
5,150%,1267.128499,27.623248,Texas,New York,39
6,125%,1055.940415,23.019374,China,Finland,569
7,125%,1055.940415,23.019374,China,New York,248
8,125%,1055.940415,23.019374,Germany,Finland,111
9,125%,1055.940415,23.019374,Germany,New York,220


In [194]:
def format_price(price:str):
    price = price[1:]
    price = price.replace(",", "").split(".")[0]
    return int(price)

![](resources/images/blJg20D.jpg)

## This function above is for choose_best_market function to choose what is the best machine --> For now I will just use what is the cheapest machine available

In [216]:
def choose_best_market(market_list:list):
    """
    Best machine market is chosen based on real price which includes shipping price --> There needs to be custom weights on markets lists with saying that item will be in stock within certain time --> Best way to calculate these weights would be to use coin profit prediction algorithms to determine how much profit will said machine lose each month and use that information to make weights --> but because I don't have that information yet I need to make my own custom weights. Let's say that machine is outdate in 3years to 0 profit and with average ROI of 1.5 years we can access how much they lose profit per one month. Let's also assume mining profit per month loss is linear -->
    Total profit: machine_price * 2
    One month of 3 years(one month profit loss): 1/(12*2)
    At 1.5year machine making 50% of original profit
    :param market_list:
    :return:
    """
    stock_name_dict = {'In stock',
     'In stock(10 \\ndays\\n)',
     'In stock(12 \\ndays\\n)',
     'In stock(15 \\ndays\\n)',
     'In stock(2 \\ndays\\n)',
     'In stock(3 \\ndays\\n)',
     'In stock(30 \\ndays\\n)',
     'In stock(5 \\ndays\\n)',
     'In stock(7 \\ndays\\n)',
     'Out of stock',
     'Pre-order(Aug\\xa02022)',
     'Pre-order(Jul\\xa02022)',
     'Pre-order(Jun\\xa02022)',
     'Pre-order(May\\xa02022)',
     'Pre-order(Oct\\xa02022)',
     'Used'}

    for market in market_list:
        if market["continent"] != "EU" or market["country"] == "United Kingdom":
            market["tax_added_price"] = market["real_price"] * 1.24
        else:
            market["tax_added_price"] = market["real_price"]

    market_list.sort(key = lambda x: x["tax_added_price"])

    """for market in market_list:
        print(market)
    print()"""
    for market in market_list:
        if market["stock"] != "Used" or market["stock"] != "Out of stock":
            return market

In [217]:
fin_modified_market = []
#for x in range(10):     # fin_market_df.shape[0]    # TODO now use all the info below to calculate total_price with shipment and taxes included!!!
for x in range(fin_market_df.shape[0]):
    machine_market_data = fin_market_df.iloc[x, 1]
    machine_name = fin_market_df.iloc[x, 0]
    all_markets_list = []

    for market in machine_market_data:
        country = market["country"]
        temp_dict = {
            "store_name": market["store_name"],
            "non_ship_price": format_price(market["price"]),    # Formatting price
            "country": country,
            "stock": market["stock"],
            "isFreeShipping": market["isFreeShipping"]
        }
        machine_size_category = check_machine_category(machine_name)
        temp_dict["size_category"] = machine_size_category
        if country in country_mapping["EU"]:
            temp_dict["continent"] = "EU"
        elif country in country_mapping["US"]:
            temp_dict["continent"] = "US"
        elif country in country_mapping["ASIA"]:
            temp_dict["continent"] = "ASIA"

        # print(temp_dict)
        shipment_cost = calculate_shipment(temp_dict, "Finland")
        # print(temp_dict)
        temp_dict["shipment_cost"] = shipment_cost
        temp_dict["real_price"] = float(temp_dict["non_ship_price"]) + float(temp_dict["shipment_cost"])
        # print(temp_dict)
        # print()
        if temp_dict["stock"] == "In stock":
            all_markets_list.append(temp_dict)

    cheapest_market = choose_best_market(all_markets_list)
    machine_market = {"machine_name": machine_name, "cheapest_market": cheapest_market, "cheapest_price": cheapest_market["real_price"] if cheapest_market is not None else 0}
    fin_modified_market.append(machine_market)
    # print(cheapest_market)
    # print()

{'store_name': 'Mining Store 5111', 'non_ship_price': 20366, 'country': 'Hong Kong', 'stock': 'In stock', 'isFreeShipping': True, 'size_category': '150%', 'continent': 'ASIA', 'shipment_cost': 0, 'real_price': 20366.0, 'tax_added_price': 25253.84}
{'store_name': 'NHASH', 'non_ship_price': 20499, 'country': 'Hong Kong', 'stock': 'In stock', 'isFreeShipping': False, 'size_category': '150%', 'continent': 'ASIA', 'shipment_cost': 653, 'real_price': 21152.0, 'tax_added_price': 26228.48}
{'store_name': 'Belay Mining', 'non_ship_price': 20995, 'country': 'United States', 'stock': 'In stock', 'isFreeShipping': False, 'size_category': '150%', 'continent': 'US', 'shipment_cost': 194, 'real_price': 21189.0, 'tax_added_price': 26274.36}
{'store_name': 'BT-Miners', 'non_ship_price': 20998, 'country': 'United States', 'stock': 'In stock', 'isFreeShipping': False, 'size_category': '150%', 'continent': 'US', 'shipment_cost': 194, 'real_price': 21192.0, 'tax_added_price': 26278.079999999998}
{'store_na

In [203]:
pd.DataFrame(fin_modified_market)

,machine_name,cheapest_market,cheapest_price
0,iPollo V1,"{'store_name': 'Mining Store 5111', 'non_ship_...",20366.0
1,Jasminer X4,"{'store_name': 'ViperaTech', 'non_ship_price':...",21194.0
2,Bitmain Antminer E9 (2.4Gh),"{'store_name': 'Casa Miners', 'non_ship_price'...",12149.0
3,Innosilicon A11 Pro ETH (1500Mh),"{'store_name': 'MillionMiner', 'non_ship_price...",9087.0
4,iPollo V1 Classic,"{'store_name': 'NHASH', 'non_ship_price': 7899...",7899.0
...,...,...,...
179,StrongU STU-U2,None,0.0
180,MicroBT Whatsminer M10S,None,0.0
181,Todek Toddminer C1 PRO,None,0.0
182,Hummer Miner Mars H1,None,0.0


In [201]:
fin_market_df

,machine_name,available_stores
0,iPollo V1,"[{'store_name': 'CoinMining Central', 'url': '..."
1,Jasminer X4,"[{'store_name': 'Safety Miner', 'url': 'https:..."
2,Bitmain Antminer E9 (2.4Gh),"[{'store_name': 'CoinMining Central', 'url': '..."
3,Innosilicon A11 Pro ETH (1500Mh),"[{'store_name': 'NHASH', 'url': 'https://www.n..."
4,iPollo V1 Classic,"[{'store_name': 'Belay Mining', 'url': 'https:..."
...,...,...
243,StrongU STU-U2,"[{'store_name': 'CoinMining Central', 'url': '..."
244,MicroBT Whatsminer M10S,"[{'store_name': 'Bitcoin Merch', 'url': 'https..."
251,Todek Toddminer C1 PRO,"[{'store_name': 'CoinMining Central', 'url': '..."
255,Hummer Miner Mars H1,"[{'store_name': 'Mining Store 5111', 'url': 'h..."


In [200]:
country_mapping

{'EU': ['Italia', 'United Kingdom', 'Germany'],
 'US': ['United States', 'Canada'],
 'ASIA': ['Israel', 'Hong Kong', 'China', 'Honk Kong']}